In [8]:
! pip install plotly
! pip install umap-learn

In [26]:
import numpy as np
import os

In [29]:
def read_embeddings(artifact_uri, dataset):
    
    print(os.path.join(artifact_uri, f'{dataset}_enrolled_embeddings.npy'))
    enrolled_embeddings = np.load(os.path.join(artifact_uri, f'{dataset}_enrolled_embeddings.npy'))
    enrolled_labels = np.load(os.path.join(artifact_uri, f'{dataset}_enrolled_labels.npy'))
    enrolled_scan_ids = np.load(os.path.join(artifact_uri, f'{dataset}_enrolled_scan_ids.npy'))
    enrolled_perspectives = np.load(os.path.join(artifact_uri, f'{dataset}_enrolled_perspectives.npy'))
    query_embeddings = np.load(os.path.join(artifact_uri, f'{dataset}_query_embeddings.npy'))
    query_labels = np.load(os.path.join(artifact_uri, f'{dataset}_query_labels.npy'))
    query_scan_ids = np.load(os.path.join(artifact_uri, f'{dataset}_query_scan_ids.npy'))
    query_perspectives = np.load(os.path.join(artifact_uri, f'{dataset}_query_perspectives.npy'))
    distances = np.load(os.path.join(artifact_uri, f'{dataset}_distances.npy'))
    
    
    return (enrolled_embeddings, enrolled_labels, enrolled_scan_ids, enrolled_perspectives, 
            query_embeddings, query_labels, query_scan_ids, query_perspectives, distances)

In [1]:
#artifact_uri = "F:\\Face\\HM_IDENT_3DFR\\src\\log\\mlruns\\633479264740573887\\7850d734dc134028ba19e52e3f37a601\\artifacts\\embeddings"
artifact_uri = "F:\\Face\\HM_IDENT_3DFR\\src\\log\\mlruns\\633479264740573887\\3926b563c310467a81f67ef50d22af6d\\artifacts\\embeddings"
dataset = 'rgb_bellus'
enrolled_embeddings, enrolled_labels, enrolled_scan_ids, enrolled_perspectives, query_embeddings, query_labels, query_scan_ids, query_perspectives, distances = read_embeddings(artifact_uri, dataset)
print(enrolled_embeddings)

NameError: name 'read_embeddings' is not defined

In [ ]:
from sklearn.manifold import TSNE
import time

start = time.time()
tsne = TSNE(n_components = 3, random_state=0)
projections = tsne.fit_transform(enrolled_embeddings)
end = time.time()
print(f"generating projections with T-SNE took: {(end-start):.2f} sec")

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import numpy as np

from typing import Dict
from pathlib import Path
from IPython.core.display import display, HTML

from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *
init_notebook_mode(connected=True)


def display_projections(
    labels: np.ndarray,
    projections: np.ndarray,
    #image_paths: np.ndarray,
    #image_data_uris: Dict[str, str],
    show_legend: bool = False,
    show_markers_with_text: bool = True
) -> None:
    # Create a separate trace for each unique label
    unique_labels = np.unique(labels)
    traces = []
    for unique_label in unique_labels:
        mask = labels == unique_label
        #customdata_masked = image_paths[mask]
        trace = go.Scatter3d(
            x=projections[mask][:, 0],
            y=projections[mask][:, 1],
            z=projections[mask][:, 2],
            mode='markers+text' if show_markers_with_text else 'markers',
            text=labels[mask],
            #customdata=customdata_masked,
            name=str(unique_label),
            marker=dict(size=8),
            hovertemplate="<b>class: %{text}</b><br>path: %{customdata}<extra></extra>"
        )
        traces.append(trace)

    # Create the 3D scatter plot
    fig = go.Figure(data=traces)
    fig.update_layout(
        scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'),
        width=1000,
        height=1000,
        showlegend=show_legend
    )

    # Convert the chart to an HTML div string and add an ID to the div
    plotly_div = fig.to_html(full_html=False, include_plotlyjs=False, div_id="scatter-plot-3d")


    # Create an HTML template including the chart div and JavaScript code
    html_template = f"""
    <!DOCTYPE html>
    <html>
        <head>
            <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
            <style>
                #image-container {{
                    position: fixed;
                    top: 0;
                    left: 0;
                    width: 200px;
                    height: 200px;
                    padding: 5px;
                    border: 1px solid #ccc;
                    background-color: white;
                    z-index: 1000;
                    box-sizing: border-box;
                    display: flex;
                    align-items: center;
                    justify-content: center;
                    text-align: center;
                }}
                #image-display {{
                    width: 100%;
                    height: 100%;
                    object-fit: contain;
                }}
            </style>
        </head>        
        <body>
            {plotly_div}
        </body>
    </html>
    """

    # Display the HTML template in the Jupyter Notebook
    display(HTML(html_template))

In [ ]:
display_projections(
    labels=enrolled_labels,
    projections=projections
    #image_paths=image_paths,
    #image_data_uris=image_data_uris
)

In [ ]:
import umap

start = time.time()
projections = umap.UMAP(n_components=3).fit_transform(enrolled_embeddings)
end = time.time()
print(f"generating projections with UMAP took: {(end-start):.2f} sec")

display_projections(
    labels=enrolled_labels,
    projections=projections
)

In [ ]:
def normalize(vec: np.ndarray):
    return vec / np.linalg.norm(vec, axis=1, keepdims=True)

In [ ]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd
from sklearn.preprocessing import normalize

# Sample Data (for illustration, replace with your actual data)
# enrolled_embeddings = np.random.rand(10, 128)  # Replace with your actual embeddings
# enrolled_labels = ['Label1', 'Label2', 'Label1', 'Label3', 'Label2', 'Label1', 'Label3', 'Label2', 'Label3', 'Label1']  # Replace with your actual labels

# Normalize embeddings
normalized_embeddings = normalize(enrolled_embeddings)

# Combine embeddings and labels into a DataFrame
df = pd.DataFrame(normalized_embeddings)
df['label'] = enrolled_labels

# Extract the sorted embeddings and labels
sorted_df = df.sort_values(by='label').reset_index(drop=True)
sorted_embeddings = sorted_df.drop(columns='label').values
sorted_labels = sorted_df['label'].values

# Compute cosine similarity matrix for sorted embeddings
cosine_sim_matrix = np.dot(sorted_embeddings, sorted_embeddings.T)

# Prepare unique labels for the axis
unique_labels = []
unique_tickvals = []
for idx, label in enumerate(sorted_labels):
    if label not in unique_labels:
        unique_labels.append(label)
        unique_tickvals.append(idx)

# Plot the heatmap
heatmap = go.Heatmap(z=cosine_sim_matrix, colorscale='Viridis')

# Setup figure with labels on the axes
fig = go.Figure(data=heatmap)

fig.update_layout(
    title="Cosine Similarity Matrix",
    xaxis_title="Image Index",
    yaxis_title="Image Index",
    font=dict(size=14),
    width=1000,
    height=1000,
    margin=dict(t=100, r=100, b=100, l=100),
    xaxis=dict(
        tickmode='array',
        tickvals=unique_tickvals,
        ticktext=unique_labels,
        tickangle=90
    ),
    yaxis=dict(
        tickmode='array',
        tickvals=unique_tickvals,
        ticktext=unique_labels
    ),
)

fig.show()